In [ ]:
!pip install --upgrade --no-cache-dir git+https://www.github.com/ekoshv/Portfolio_Optim.git@NN_V29
!pip install scikit-learn-intelex
#!pip install --upgrade --no-cache-dir tensorflow
!pip install tensorflow-addons
!pip install dill
!pip install pathos

url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3

import numpy as np
import pandas as pd
from ekoptim import ekoptim
import datetime
from tqdm import tqdm
import traceback
from sklearn.covariance import LedoitWolf
import dill
import types
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Assuming you've uploaded 'workspace.pkl' to your Google Drive
file_path = '/content/drive/My Drive/Colab_Files/data_20230712_011152.pkl'
# Load workspace
with open(file_path, 'rb') as f:
    data = dill.load(f)

# Now you can access your variables as:
optimizerTV = data['optimizerTV']
rates_TV = data['rates_TV']
rates_MT5 = data['rates_MT5']
selected_symb = data['selected_symb']
from ekoptim import ekoptim

Dqp = 32 # past days for deep learning
Dyp = 16 # past days (not used)
Dyf = 14 # future days
n_t = 2 # tile size
xhh = 0.15 # Percentage Threshold
xhl = 0.53*xhh
xlh = -xhl
xll = -xhh
optimizerTV.Prepare_Data(tile_size=(n_t,int(n_t*Dqp/4)),xrnd=1e-3,#(n*Dqp->m=n*Dqp/4)
                          Selected_symbols=selected_symb,
                          raw_data=False,
                          Dqp=Dqp, Dyp=Dyp, Dyf=Dyf, Thi=1,
                          hh=xhh,#
                          hl=xhl,#
                          lh=xlh,#
                          ll=xll, test=False) #None

optimizerTV.NNmake(inps_select = [0,1,7], learning_rate=0.001, epochs=300, batch_size=32,
                   model_simple=False, val_size=0.33,
                   k_n=None, f1_method='macro', f1_w=False, mcc_w=False, filters=256, dSize=1024,
                   load_train=False,
                   back_test_en =True, bt_ratio=0.1, back_test_as_val_en=False)

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
date_time = now.strftime("%Y%m%d_%H%M%S")

zipf = zipfile.ZipFile(f'logs_and_checkpoints.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('logs/', zipf)
zipdir('checkpoints/', zipf)
zipf.close()

!cp logs_and_checkpoints.zip '/content/drive/My Drive/Colab_Files/'

#optimizerTV.load_model_fit()
#optimizerTV.predict_all(inps_select = [0,1,2], filters=64)
#betax = optimizerTV.Predicted_Rates

#xeta = pd.DataFrame([(x[0].squeeze(),x[1].squeeze()) for x in betax])
#xeta

In [ ]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
date_time = now.strftime("%Y%m%d_%H%M%S")

zipf = zipfile.ZipFile(f'logs_and_checkpoints_xyz.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('logs/', zipf)
zipdir('checkpoints/', zipf)
zipf.close()

!cp logs_and_checkpoints_xyz.zip '/content/drive/My Drive/Colab_Files/'